# 处理环境


在将脚本作为 Azure 机器学习作业运行时，需要定义该作业运行的执行上下文。 一个关键配置是将运行脚本的计算目标。 这可以是本地工作站（在本例中为计算实例），也可以是远程计算目标，例如按需预配的 Azure 机器学习托管计算群集。

在此笔记本中，你将创建计算群集并浏览作业的计算目标。

## 准备工作

需要最新版本的 azureml-ai-ml 包才能运行此笔记本中的代码。 运行下面的单元以验证是否已安装它。

> **注意**：
> 如果未安装 azure-ai-ml 包，请运行 `pip install azure-ai-ml` 以进行安装。

In [ ]:
## 连接到工作区

安装必需的 SDK 包后，就可以连接到工作区了。

若要连接到工作区，我们需要标识符参数 - 订阅 ID、资源组名称和工作区名称。 已为你填写资源组名称和工作区名称。 只需订阅 ID 即可完成命令。

若要查找所需的参数，请单击工作室右上角的订阅和工作区名称。 右侧将打开一个窗格。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 复制订阅 ID，并将“YOUR-SUBSCRIPTION-ID”替换为复制的值。 </p>

## 将脚本作为作业运行

若要训练模型，请先在 src 文件夹中创建 diabetes_training.py 脚本 。 该脚本使用训练数据所在的文件夹中的 diabetes.csv 文件。

请注意，在脚本的开头导入库。 这些库中的函数用于处理数据和训练模型。 无论使用什么计算来运行脚本，都必须安装这些库。

In [ ]:
创建脚本后，可以将脚本作为作业运行。 该脚本使用通用库。 因此，你可以使用包括 pandas、numpy 和 scikit-learn 等的策展环境。

作业使用最新版本的策展环境：`AzureML-sklearn-0.24-ubuntu18.04-py37-cpu`。

In [ ]:
当作业正在运行时，你已可以运行下一个单元。

## 列出环境

一起浏览工作区中的环境。 

在上一个作业中，你使用了一个策展环境。 若要浏览工作区中已存在的所有环境，可以列出环境： 

In [ ]:
请注意，所有策展环境的名称都以 AzureML- 开头（不能将此前缀用于自己的环境）。

若要查看特定环境，可以按其名称和版本检索环境。 例如，可以检索用于上一个作业的策展环境的说明和标记 ：

## 创建和使用自定义环境

如果策展环境不包含运行脚本所需的所有 Python 包，则可以创建自己的自定义环境。 通过列出环境中所有必要的包，可以轻松重新运行脚本。 所有依赖项都存储在环境中，然后你在作业配置中指定所需环境即可，不用管所使用的计算。

例如，只需从 Docker 映像创建环境即可。 某些框架（如 PyTorch）将具有一个公共 Docker 映像，该映像已包含所需的一切。 

让我们从 Docker 映像创建环境：

In [ ]:
现已在工作区中注册该环境，可以在将脚本作为作业运行时引用它：

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 作业很快就会失败！ 查看错误消息。 </p>

错误消息会告知你没有名为 pandas 的模块。 出现此类错误的可能原因有两个：

- 脚本使用 pandas，但未导入库 (`import pandas as pd`)。 
- 脚本确实在脚本顶部导入了库，但计算未安装库 (`pip install pandas`)。

查看 `diabetes-training.py` 脚本后，可以注意到脚本是正确的，这意味着未安装库。 换句话说，环境不包含必要的包。

使用上一作业中用到的基本 Docker 映像创建新环境。 现在，你将添加 conda 规范，以确保安装必要的包。 首先，运行以下单元以创建 conda 规范文件：

请注意，所有必需的依赖项都包含在 conda 规范文件中，以便脚本成功运行。

使用基础 Docker 映像和 conda 规范文件创建新环境，以添加必要的依赖项。 Azure 机器学习将基于提供的 Docker 映像生成 conda 环境。 

In [ ]:
现在，可以使用新环境提交作业来运行脚本：

使用新的自定义环境提交作业会触发环境生成。 首次使用新创建的环境时，生成环境可能需要 10-15 分钟，这也意味着作业需要更长的时间才能完成。 

还可以选择先手动触发环境生成，再提交作业。 只需在首次使用时生成环境。 

当作业触发新环境生成时，可以在作业的“输出 + 日志”选项卡中查看生成日志。 打开 azureml-logs/20_image_build_log.txt 以检查环境生成日志。 

![生成日志屏幕截图](./images/screenshot-logs.png)

In [ ]:
env = ml_client.environments.get("AzureML-sklearn-0.24-ubuntu18.04-py37-cpu", version=44)
print(env. description, env.tags)

## Create and use a custom environment

If a curated environment doesn't include all the Python packages you need to run your script, you can create your own custom environment. By listing all necessary packages in an environment, you can easily re-run your scripts. All the dependencies are stored in the environment which you can then specify in the job configuration, independent of the compute you use.

For example, you can create an environment simply from a Docker image. Certain frameworks like PyTorch will have a public Docker image that already includes everything you need. 

Let's create an environment from a Docker image:

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_image = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    name="docker-image-example",
    description="Environment created from a Docker image.",
)
ml_client.environments.create_or_update(env_docker_image)

The environment is now registered in your workspace and you can reference it when you run a script as a job:

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="docker-image-example:1",
    compute="aml-cluster",
    display_name="diabetes-train-custom-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> The job will quickly fail! Review the error message. </p>

The error message will tell you that there is no module named pandas. There are two possible causes for such an error:

- The script uses pandas but didn't import the library (`import pandas as pd`). 
- The script does import the library at the top of the script but the compute didn't have the library installed (`pip install pandas`).

After reviewing the `diabetes-training.py` script you can observe the script is correct, which means the library wasn't installed. In other words, the environment didn't include the necessary packages.

Let's create a new environment, using the base Docker image used in the previous job. Now, you'll add a conda specification to ensure the necessary packages will be installed. First, run the following cell to create the conda specification file:

In [ ]:
%%writefile src/conda-env.yml
name: basic-env-cpu
channels:
  - conda-forge
dependencies:
  - python=3.7
  - scikit-learn
  - pandas
  - numpy
  - matplotlib

Note that all necessary dependencies are included in the conda specification file for the script to run successfully.

Create a new environment using the base Docker image **and** the conda specification file to add the necessary dependencies. Azure Machine Learning will build the conda environment on top of the Docker image you provided. 

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="./src/conda-env.yml",
    name="docker-image-plus-conda-example",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

Now, you can submit a job with the new environment to run the script:

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="docker-image-plus-conda-example:1",
    compute="aml-cluster",
    display_name="diabetes-train-custom-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Submitting the job with the new custom environment triggers the build of the environment. The first time you use a newly created environment, it can take 10-15 minutes to build the environment, which also means your job will take longer to complete. 

You can also choose to manually trigger the build of the environment before you submit a job. The environment only needs to be built the first time you use it. 

When the job triggers the build of a new environment, you can review the logs of the build in the **Outputs + logs** tab of the job. Open **azureml-logs/20_image_build_log.txt** to inspect the logs of the environment build. 

![Screenshot build logs](./images/screenshot-logs.png)